<a href="https://colab.research.google.com/github/Tharungovind/GOVINDTHARUN_INFO5731_FALL2024/blob/main/govind_tharun_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
# Your code here

import requests
from bs4 import BeautifulSoup
import time

def get_reviews(movie_id, max_reviews=1000):
    reviews = []
    page = 0
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    while len(reviews) < max_reviews:
        url = f"https://www.imdb.com/title/{movie_id}/reviews/_ajax?paginationKey="
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        review_blocks = soup.find_all('div', class_='text show-more__control')

        for review_block in review_blocks:
            reviews.append(review_block.get_text())
            if len(reviews) % 100 == 0 :
               print(len(reviews))

            if len(reviews) >= max_reviews:
                break

        pagination_key = soup.find('div', {'data-key': True})

        if pagination_key:
            page = pagination_key['data-key']
        else:
            break

        time.sleep(2)

    return reviews

movies = [
    {"title": "Oppenheimer", "id": "tt15398776"},
]

all_reviews = {}

for movie in movies:
    print(f"Collecting reviews for {movie['title']}...")
    reviews = get_reviews(movie['id'], max_reviews=1000)
    all_reviews[movie['title']] = reviews
    print(f"Collected {len(reviews)} reviews for {movie['title']}")

for movie, reviews in all_reviews.items():
    print(f"{movie}: {len(reviews)} reviews")

import csv

for movie, reviews in all_reviews.items():
    with open(f"{movie}_reviews.csv", 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Review'])
        for review in reviews:
            writer.writerow([review])


100
200
300
400
500
600
700
800
900
1000
Collected 1000 reviews for Oppenheimer
Oppenheimer: 1000 reviews


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:
# Write code for each of the sub parts with proper comments.
!pip install pandas nltk


In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


nltk.download('stopwords')
nltk.download('wordnet')

# Load the CSV file
df = pd.read_csv('/content/Oppenheimer_reviews.csv')

print("Columns in the DataFrame:", df.columns)
print(df.head())

# Strip whitespace from column names
df.columns = df.columns.str.strip()

def clean_text(text):
    if pd.isna(text):
        return ''

    # (1) Remove noise (special characters and punctuations)
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    ps = PorterStemmer()
    text = ' '.join(ps.stem(word) for word in text.split())

    # (6) Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())

    return text

df['cleaned_text'] = df['Review'].apply(clean_text)

df.to_csv('cleaned_data.csv', index=False)
print("Data cleaned and saved successfully.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Columns in the DataFrame: Index(['Review'], dtype='object')
                                              Review
0  You'll have to have your wits about you and yo...
1  One of the most anticipated films of the year ...
2  I'm a big fan of Nolan's work so was really lo...
3  I'm still collecting my thoughts after experie...
4  "Oppenheimer" is a biographical thriller film ...
Data cleaned and saved successfully.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [4]:
# Your code here
import spacy
import pandas as pd
from collections import Counter

# Load the spacy model for English language processing
nlp = spacy.load("en_core_web_sm")

df = pd.read_csv('/content/cleaned_data.csv')

def pos_tag(text):
    doc = nlp(text)
    pos_counts = Counter([token.pos_ for token in doc])

    # Count nouns, verbs, adjectives, and adverbs
    noun_count = pos_counts.get('NOUN', 0)
    verb_count = pos_counts.get('VERB', 0)
    adj_count = pos_counts.get('ADJ', 0)
    adv_count = pos_counts.get('ADV', 0)

    return noun_count, verb_count, adj_count, adv_count

df['Nouns'], df['Verbs'], df['Adjectives'], df['Adverbs'] = zip(*df['cleaned_text'].apply(pos_tag))

def parsing_analysis(text):
    doc = nlp(text)

    print("Dependency Parsing:")
    for token in doc:
        print(f'{token.text} -> {token.dep_} (Head: {token.head.text})')

    print("\nConstituency Parsing:")
    for token in doc:
        print(f'{token.text} ({token.pos_}) --> {token.dep_}')

example_sentence = df['cleaned_text'].iloc[0]
print(f"\nExample Sentence: {example_sentence}")
parsing_analysis(example_sentence)

def extract_named_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

df['Entities'] = df['cleaned_text'].apply(extract_named_entities)

entity_counts = Counter()
for entities in df['Entities']:
    for ent in entities:
        entity_counts[ent[1]] += 1

print("\nNamed Entity Recognition Counts:")
for entity_type, count in entity_counts.items():
    print(f'{entity_type}: {count}')

df.to_csv('syntax_structure_analysis.csv', index=False)
print("Syntax and structure analysis saved to 'syntax_structure_analysis.csv'")




Example Sentence: youll wit brain fulli switch watch oppenheim could easili get away nonattent viewer intellig filmmak show audienc great respect fire dialogu pack inform relentless pace jump differ time oppenheim life continu hour runtim visual clue guid viewer time youll get grip quit quickli relentless help express urgenc u attack chase atom bomb germani could absolut career best perform consistenli brilliant cillian murphi anchor film nail oscar perform fact whole cast fantast apart mayb sometim overwrought emili blunt perform rdj also particularli brilliant return proper act decad call screenplay den layer id say thick bibl cinematographi quit stark spare part imbu rich luciou colour moment especi scene florenc pugh score beauti time mostli anxiou oppress ad relentless pace hour runtim fli found intens tax highli reward watch film make finest realli great watch
Dependency Parsing:
you -> nsubj (Head: wit)
ll -> aux (Head: wit)
wit -> nsubj (Head: absolut)
brain -> compound (Head:

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [ ]:
/content/syntax_structure_analysis.csv

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below

#I really enjoyed in executing the code for this assignment as it involved from Web scraping to Text feature . I got some difficulties in executing the code i went to lectures and able to execute the code. the time provided for execution of code is not sufficient for me as extension of time helped for me to complete thhe assignment